In [1]:
! pip install pandas tqdm sacrebleu transformers torch radon


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 4.5 MB/s eta 0:00:00


In [10]:
import pandas as pd
import radon.complexity as cc
from radon.metrics import mi_visit
from sacrebleu.metrics import BLEU
import math

# Load dataset (update column names as per your CSV)
df = pd.read_csv("bug_diffs_with_llm.csv")

# Initialize BLEU scorer
bleu_scorer = BLEU(effective_order=True)

# Function to compute radon metrics (only for Python files)
def compute_radon_metrics(source_code: str):
    if not isinstance(source_code, str) or not source_code.strip():
        return float("nan"), float("nan"), float("nan")
    try:
        mi = mi_visit(source_code, True)
        loc = len([line for line in source_code.split("\n") if line.strip()])
        complexity_list = cc.cc_visit(source_code)
        cc_total = sum([block.complexity for block in complexity_list])
        return mi, cc_total, loc
    except Exception as e:
        print(f"Radon error: {e}")
        return float("nan"), float("nan"), float("nan")

# Apply without Filename
df["MI_Before"], df["CC_Before"], df["LOC_Before"] = zip(
    *df.apply(lambda row: compute_radon_metrics(row["source_before"]), axis=1)
)

df["MI_After"], df["CC_After"], df["LOC_After"] = zip(
    *df.apply(lambda row: compute_radon_metrics(row["source_current"]), axis=1)
)


# Compute changes
df["MI_Change"] = df["MI_After"] - df["MI_Before"]
df["CC_Change"] = df["CC_After"] - df["CC_Before"]
df["LOC_Change"] = df["LOC_After"] - df["LOC_Before"]

# Compute BLEU token similarity
def safe_bleu(before, after):
    if not isinstance(before, str) or not isinstance(after, str):
        return 0.0
    before_tokens = before.split()
    after_tokens = after.split()
    if not before_tokens or not after_tokens:
        return 0.0
    score = bleu_scorer.sentence_score(" ".join(after_tokens), [" ".join(before_tokens)]).score
    return score / 100.0  # normalize to [0,1]

df["Token_Similarity"] = df.apply(
    lambda row: safe_bleu(row["source_before"], row["source_current"]), axis=1
)

# Classification
TOKEN_THRESHOLD = 0.75
df["Classification_Token"] = df["Token_Similarity"].apply(
    lambda x: "Minor" if x >= TOKEN_THRESHOLD else "Major"
)

# Save results
df.to_csv("lab_3.csv", index=False)

print(" Processing complete. Saved as lab_3.csv")


Radon error: unterminated string literal (detected at line 4) (<unknown>, line 4)
Radon error: multiple exception types must be parenthesized (<unknown>, line 429)
Radon error: unterminated string literal (detected at line 8) (<unknown>, line 8)
Radon error: invalid decimal literal (<unknown>, line 31)
Radon error: unterminated string literal (detected at line 27) (<unknown>, line 27)
Radon error: unterminated string literal (detected at line 46) (<unknown>, line 46)
Radon error: unexpected indent (<unknown>, line 2)
Radon error: unterminated string literal (detected at line 6) (<unknown>, line 6)
Radon error: unterminated string literal (detected at line 17) (<unknown>, line 17)
Radon error: unterminated string literal (detected at line 8) (<unknown>, line 8)
Radon error: unterminated string literal (detected at line 17) (<unknown>, line 17)
Radon error: multiple exception types must be parenthesized (<unknown>, line 483)
Radon error: unterminated string literal (detected at line 27) 

<unknown>:936: SyntaxWarning: invalid escape sequence '\s'
<unknown>:957: SyntaxWarning: invalid escape sequence '\s'
<unknown>:24: SyntaxWarning: invalid escape sequence '\s'
<unknown>:24: SyntaxWarning: invalid escape sequence '\s'
<unknown>:963: SyntaxWarning: invalid escape sequence '\s'
<unknown>:24: SyntaxWarning: invalid escape sequence '\s'
<unknown>:24: SyntaxWarning: invalid escape sequence '\s'
<unknown>:24: SyntaxWarning: invalid escape sequence '\s'
<unknown>:24: SyntaxWarning: invalid escape sequence '\s'
<unknown>:975: SyntaxWarning: invalid escape sequence '\s'
<unknown>:975: SyntaxWarning: invalid escape sequence '\s'
<unknown>:988: SyntaxWarning: invalid escape sequence '\s'
<unknown>:988: SyntaxWarning: invalid escape sequence '\s'
<unknown>:989: SyntaxWarning: invalid escape sequence '\s'
<unknown>:1011: SyntaxWarning: invalid escape sequence '\s'
<unknown>:24: SyntaxWarning: invalid escape sequence '\s'
<unknown>:24: SyntaxWarning: invalid escape sequence '\s'
<unk

Radon error: multiple exception types must be parenthesized (<unknown>, line 347)
Radon error: invalid syntax (<unknown>, line 1103)
Radon error: multiple exception types must be parenthesized (<unknown>, line 46)
Radon error: multiple exception types must be parenthesized (<unknown>, line 347)
Radon error: invalid syntax (<unknown>, line 1)
Radon error: multiple exception types must be parenthesized (<unknown>, line 104)
Radon error: invalid decimal literal (<unknown>, line 52)
Radon error: invalid syntax (<unknown>, line 1206)
Radon error: multiple exception types must be parenthesized (<unknown>, line 347)
Radon error: multiple exception types must be parenthesized (<unknown>, line 373)
Radon error: invalid decimal literal (<unknown>, line 52)
Radon error: Missing parentheses in call to 'print'. Did you mean print(...)? (<unknown>, line 62)
Radon error: multiple exception types must be parenthesized (<unknown>, line 351)
Radon error: multiple exception types must be parenthesized (<

<unknown>:1168: SyntaxWarning: invalid escape sequence '\s'
<unknown>:1216: SyntaxWarning: invalid escape sequence '\s'
<unknown>:1227: SyntaxWarning: invalid escape sequence '\s'
<unknown>:1227: SyntaxWarning: invalid escape sequence '\s'
<unknown>:1233: SyntaxWarning: invalid escape sequence '\s'
<unknown>:1238: SyntaxWarning: invalid escape sequence '\s'
<unknown>:1238: SyntaxWarning: invalid escape sequence '\s'
<unknown>:1238: SyntaxWarning: invalid escape sequence '\s'
<unknown>:1238: SyntaxWarning: invalid escape sequence '\s'


Radon error: invalid syntax (<unknown>, line 1264)
Radon error: invalid syntax (<unknown>, line 1264)
Radon error: multiple exception types must be parenthesized (<unknown>, line 186)
Radon error: invalid syntax (<unknown>, line 1)
Radon error: unterminated string literal (detected at line 77) (<unknown>, line 77)
Radon error: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (<unknown>, line 13)
Radon error: multiple exception types must be parenthesized (<unknown>, line 334)
Radon error: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (<unknown>, line 13)
Radon error: multiple exception types must be parenthesized (<unknown>, line 334)
Radon error: invalid syntax (<unknown>, line 1)
Radon error: invalid syntax (<unknown>, line 6)
Radon error: invalid syntax (<unknown>, line 158)
Radon error: invalid syntax (<unknown>, line 1302)
Radon error: multiple exception types must be parenthesized (

...

Radon error: unterminated string literal (detected at line 28) (<unknown>, line 28)
Radon error: invalid decimal literal (<unknown>, line 18)
Radon error: unexpected character after line continuation character (<unknown>, line 1)
Radon error: unexpected character after line continuation character (<unknown>, line 1)
Radon error: invalid decimal literal (<unknown>, line 22)
Radon error: unterminated string literal (detected at line 14) (<unknown>, line 14)
Radon error: unterminated string literal (detected at line 26) (<unknown>, line 26)
Radon error: invalid syntax (<unknown>, line 1)
Radon error: invalid syntax (<unknown>, line 5)
Radon error: invalid syntax (<unknown>, line 2)
Radon error: unterminated string literal (detected at line 13) (<unknown>, line 13)
Radon error: invalid decimal literal (<unknown>, line 25)
Radon error: unterminated string literal (detected at line 14) (<unknown>, line 14)
Radon error: unterminated string literal (detected at line 16) (<unknown>, line 16)
Ra

<unknown>:1683: SyntaxWarning: invalid escape sequence '\s'
<unknown>:1683: SyntaxWarning: invalid escape sequence '\s'
<unknown>:1683: SyntaxWarning: invalid escape sequence '\s'
<unknown>:1683: SyntaxWarning: invalid escape sequence '\s'


Radon error: invalid syntax (<unknown>, line 1)
Radon error: unterminated string literal (detected at line 30) (<unknown>, line 30)
Radon error: invalid decimal literal (<unknown>, line 37)
Radon error: unterminated string literal (detected at line 23) (<unknown>, line 23)
Radon error: unterminated string literal (detected at line 23) (<unknown>, line 23)
Radon error: unterminated string literal (detected at line 23) (<unknown>, line 23)


<unknown>:1683: SyntaxWarning: invalid escape sequence '\s'
<unknown>:1683: SyntaxWarning: invalid escape sequence '\s'


Radon error: unterminated string literal (detected at line 11) (<unknown>, line 11)
Radon error: invalid syntax (<unknown>, line 1)
Radon error: invalid syntax (<unknown>, line 2)
Radon error: unterminated string literal (detected at line 38) (<unknown>, line 38)
Radon error: unterminated string literal (detected at line 6) (<unknown>, line 6)
Radon error: unterminated string literal (detected at line 10) (<unknown>, line 10)
Radon error: unterminated string literal (detected at line 110) (<unknown>, line 110)
Radon error: unterminated string literal (detected at line 13) (<unknown>, line 13)
Radon error: invalid syntax (<unknown>, line 1)
Radon error: unterminated string literal (detected at line 79) (<unknown>, line 79)
Radon error: unterminated string literal (detected at line 18) (<unknown>, line 18)
Radon error: unterminated string literal (detected at line 24) (<unknown>, line 24)
Radon error: unterminated string literal (detected at line 24) (<unknown>, line 24)
Radon error: lea

In [13]:
df.tail(100)


,hash,message,filename,source_before,source_current,diff,LLM_inference,MI_Before,CC_Before,LOC_Before,MI_After,CC_After,LOC_After,MI_Change,CC_Change,LOC_Change,Token_Similarity,Classification_Token
2302,87d5f5b9a9697434e6d972b021201105eabb54e6,update project files (#5457)\n\n* update pre-c...,.github/ISSUE_TEMPLATE/config.yml,blank_issues_enabled: false\ncontact_links:\n ...,blank_issues_enabled: false\ncontact_links:\n ...,"@@ -1,11 +1,11 @@\n blank_issues_enabled: fals...",add missing link to new issue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.546004,Major
2303,87d5f5b9a9697434e6d972b021201105eabb54e6,update project files (#5457)\n\n* update pre-c...,NaN,# Security Policy\n\nIf you believe you have i...,NaN,"@@ -1,19 +0,0 @@\n-# Security Policy\n-\n-If y...",add missing comment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,Major
2304,87d5f5b9a9697434e6d972b021201105eabb54e6,update project files (#5457)\n\n* update pre-c...,.github/dependabot.yml,"version: 2\nupdates:\n- package-ecosystem: ""gi...",version: 2\nupdates:\n - package-ecosystem: g...,"@@ -1,9 +1,24 @@\n version: 2\n updates:\n-- p...",add missing missing tags,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.122904,Major
2305,87d5f5b9a9697434e6d972b021201105eabb54e6,update project files (#5457)\n\n* update pre-c...,.github/pull_request_template.md,"<!--\nBefore opening a PR, open a ticket descr...","<!--\nBefore opening a PR, open a ticket descr...","@@ -1,6 +1,7 @@\n <!--\n Before opening a PR, ...",add more comments to tox/tox-fix,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.590250,Major
2306,87d5f5b9a9697434e6d972b021201105eabb54e6,update project files (#5457)\n\n* update pre-c...,.github/workflows/lock.yaml,name: 'Lock threads'\n# Lock closed issues tha...,name: Lock inactive closed issues\n# Lock clos...,"@@ -1,8 +1,9 @@\n-name: 'Lock threads'\n-# Loc...",add more info to lock-threads example,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.778973,Minor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2397,fb54159861708558b5f5658ebdc14709d984361c,secret key rotation: fix key list ordering\n\n...,CHANGES.rst,Version 3.1.1\n-------------\n\nUnreleased\n\n...,Version 3.1.1\n-------------\n\nUnreleased\n\n...,"@@ -3,6 +3,8 @@ Version 3.1.1\n \n Unreleased\...",add missing line to unreleasednit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.998181,Minor
2398,fb54159861708558b5f5658ebdc14709d984361c,secret key rotation: fix key list ordering\n\n...,src/flask/sessions.py,from __future__ import annotations\n\nimport c...,from __future__ import annotations\n\nimport c...,"@@ -318,11 +318,12 @@ class SecureCookieSessio...",add fallback key to securecookiesessioninterface,59.214566,50.0,320.0,59.148218,50.0,321.0,-0.066349,0.0,1.0,0.995099,Minor
2399,fb54159861708558b5f5658ebdc14709d984361c,secret key rotation: fix key list ordering\n\n...,tests/test_basic.py,import gc\nimport re\nimport typing as t\nimpo...,import gc\nimport re\nimport typing as t\nimpo...,"@@ -381,14 +381,21 @@ def test_session_secret_...",add test for secret key fallbacks,0.000000,357.0,1465.0,0.000000,357.0,1472.0,0.000000,0.0,7.0,0.994983,Minor
2400,bbaf13333fd00644313488243cfe547b13dcae78,fix syntax,CHANGES.rst,Version 3.1.1\n-------------\n\nReleased 2025-...,Version 3.1.1\n-------------\n\nReleased 2025-...,"@@ -5,9 +5,9 @@ Released 2025-05-13\n \n - F...",add missing `--help` for `nice` fix,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.999639,Minor


In [12]:
df2 = pd.read_csv("bug_diffs_with_llm.csv")
df2

,hash,message,filename,source_before,source_current,diff,LLM_inference
0,3d719f35f5c1ee4ce3dc01fb2167ef49c0180cf6,"Added docs, fixed some bugs I introduced last ...",artwork/logo-full.svg,NaN,"<?xml version=""1.0"" encoding=""UTF-8"" standalon...","@@ -0,0 +1,329 @@\n+<?xml version=""1.0"" encodi...",add new example in inkscape
1,3d719f35f5c1ee4ce3dc01fb2167ef49c0180cf6,"Added docs, fixed some bugs I introduced last ...",docs/_static/flask.png,NaN,PNG\r\n\n,Binary files /dev/null and b/docs/_static/flas...,fix flask image
2,3d719f35f5c1ee4ce3dc01fb2167ef49c0180cf6,"Added docs, fixed some bugs I introduced last ...",docs/_static/logo-full.png,NaN,PNG\r\n\n,Binary files /dev/null and b/docs/_static/logo...,fix logo path
3,3d719f35f5c1ee4ce3dc01fb2167ef49c0180cf6,"Added docs, fixed some bugs I introduced last ...",docs/_templates/sidebarintro.html,NaN,<h3>About Flask</h3>\n<p>\n Flask is a micro ...,"@@ -0,0 +1,7 @@\n+<h3>About Flask</h3>\n+<p>\n...",add link to the missing word in the list
4,3d719f35f5c1ee4ce3dc01fb2167ef49c0180cf6,"Added docs, fixed some bugs I introduced last ...",docs/_templates/sidebarlogo.html,NaN,"<p class=""logo""><a href=""{{ pathto(master_doc)...","@@ -0,0 +1,3 @@\n+<p class=""logo""><a href=""{{ ...",add logo to missing missing tag
...,...,...,...,...,...,...,...
2397,fb54159861708558b5f5658ebdc14709d984361c,secret key rotation: fix key list ordering\n\n...,CHANGES.rst,Version 3.1.1\n-------------\n\nUnreleased\n\n...,Version 3.1.1\n-------------\n\nUnreleased\n\n...,"@@ -3,6 +3,8 @@ Version 3.1.1\n \n Unreleased\...",add missing line to unreleasednit
2398,fb54159861708558b5f5658ebdc14709d984361c,secret key rotation: fix key list ordering\n\n...,src/flask/sessions.py,from __future__ import annotations\n\nimport c...,from __future__ import annotations\n\nimport c...,"@@ -318,11 +318,12 @@ class SecureCookieSessio...",add fallback key to securecookiesessioninterface
2399,fb54159861708558b5f5658ebdc14709d984361c,secret key rotation: fix key list ordering\n\n...,tests/test_basic.py,import gc\nimport re\nimport typing as t\nimpo...,import gc\nimport re\nimport typing as t\nimpo...,"@@ -381,14 +381,21 @@ def test_session_secret_...",add test for secret key fallbacks
2400,bbaf13333fd00644313488243cfe547b13dcae78,fix syntax,CHANGES.rst,Version 3.1.1\n-------------\n\nReleased 2025-...,Version 3.1.1\n-------------\n\nReleased 2025-...,"@@ -5,9 +5,9 @@ Released 2025-05-13\n \n - F...",add missing `--help` for `nice` fix
